In [1]:

import ibis
from ibis import _
from cng.utils import *
from cng.h3 import * 
import os
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
install_h3()



In [ ]:
set_secrets(con, key ="", secret="", endpoint="s3-west.nrp-nautilus.io")
ncp_only = con.read_parquet("s3://public-ncp/hex/ncp_only/**")

ncp_only.head().execute()

,Z,h8,h0
0,-128,88005b2699fffff,8001fffffffffff
1,-128,88005b2699fffff,8001fffffffffff
2,-128,88005b2699fffff,8001fffffffffff
3,-128,88005b2699fffff,8001fffffffffff
4,-128,88005b2699fffff,8001fffffffffff


In [6]:
ncp_only.group_by(_.Z).count().execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Z,CountStar(ibis_read_parquet_x26gwh2wkfh5pag2vtuisxotcm)
0,0,14049748
1,11,1238564
2,16,877046
3,15,941256
4,5,2348415
5,9,1459327
6,14,1000551
7,17,806019
8,4,2600968
9,7,1859590


In [3]:
# repartition without na and rescaled
con.read_parquet("s3://public-ncp/hex/ncp_only/**").filter(_.Z != -128).mutate(ncp = _.Z / 19).drop("Z").to_parquet("s3://public-ncp/rehex/ncp_only/", partition_by = "h0")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
ncp_bio = con.read_parquet("s3://public-ncp/hex/ncp_biod_nathab/**")

#ncp_bio.head().execute()
#ncp_bio.group_by(_.Z).count().execute()

ncp_bio.mutate(ncp = _.Z / 19).drop("Z").to_parquet("s3://public-ncp/rehex/ncp_biod_nathab/", partition_by = "h0")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
import leafmap.maplibregl as leafmap

m = leafmap.Map()

m.add_cog_layer("https://minio.carlboettiger.info/public-ncp/NCP_biod_nathab_cog.tif", titiler_endpoint="https://titiler.nrp-nautilus.io")
m

Html(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass': True, 'showZoom': True, 'visual…